In [1]:
# # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# # THEN FEEL FREE TO DELETE THIS CELL.
# # NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# # ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# # NOTEBOOK.
# import kagglehub
# nelgiriyewithana_credit_card_fraud_detection_dataset_2023_path = kagglehub.dataset_download('nelgiriyewithana/credit-card-fraud-detection-dataset-2023')

# print('Data source import complete.')


# 1. Import python libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mlt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import tensorflow as tf
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

import os
file_path = ''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        file_path = os.path.join(dirname, filename)
train_df = pd.read_csv(file_path)


/kaggle/input/creditcard_2023.csv


In [3]:
%%time
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

CPU times: user 42 µs, sys: 5 µs, total: 47 µs
Wall time: 51.5 µs


# 2. Data Analysis

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568630 entries, 0 to 568629
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      568630 non-null  int64  
 1   V1      568630 non-null  float64
 2   V2      568630 non-null  float64
 3   V3      568630 non-null  float64
 4   V4      568630 non-null  float64
 5   V5      568630 non-null  float64
 6   V6      568630 non-null  float64
 7   V7      568630 non-null  float64
 8   V8      568630 non-null  float64
 9   V9      568630 non-null  float64
 10  V10     568630 non-null  float64
 11  V11     568630 non-null  float64
 12  V12     568630 non-null  float64
 13  V13     568630 non-null  float64
 14  V14     568630 non-null  float64
 15  V15     568630 non-null  float64
 16  V16     568630 non-null  float64
 17  V17     568630 non-null  float64
 18  V18     568630 non-null  float64
 19  V19     568630 non-null  float64
 20  V20     568630 non-null  float64
 21  V21     56

# - 2.1 **Observations**

- Dataset does not have null values
- Dataset have all the values in numeric/ decimal format

In [5]:
train_df.head()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,0.637735,-0.987020,0.293438,-0.941386,0.549020,1.804879,0.215598,0.512307,0.333644,0.124270,0.091202,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,0.529808,0.140107,1.564246,0.574074,0.627719,0.706121,0.789188,0.403810,0.201799,-0.340687,-0.233984,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,0.690708,-0.272985,0.659201,0.805173,0.616874,3.069025,-0.577514,0.886526,0.239442,-2.366079,0.361652,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,0.575231,-0.752581,0.737483,0.592994,0.559535,-0.697664,-0.030669,0.242629,2.178616,-1.345060,-0.378223,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,0.968046,-1.203171,1.029577,1.439310,0.241454,0.153008,0.224538,0.366466,0.291782,0.445317,0.247237,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


In [6]:
train_df.describe()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,568630.000000,5.686300e+05,5.686300e+05,5.686300e+05,568630.000000,5.686300e+05,5.686300e+05,568630.000000,568630.000000,568630.000000,5.686300e+05,568630.000000,5.686300e+05,5.686300e+05,5.686300e+05,5.686300e+05,5.686300e+05,568630.000000,568630.000000,568630.000000,5.686300e+05,5.686300e+05,5.686300e+05,5.686300e+05,5.686300e+05,5.686300e+05,568630.000000,5.686300e+05,5.686300e+05,568630.000000,568630.0
mean,284314.500000,-5.118237e-17,-1.023647e-16,1.023647e-16,0.000000,2.559118e-17,2.559118e-17,0.000000,0.000000,0.000000,1.023647e-16,0.000000,-1.535471e-16,-1.279559e-17,-5.118237e-17,3.198898e-17,5.118237e-17,0.000000,0.000000,0.000000,-2.559118e-17,1.279559e-17,1.599449e-18,6.397796e-18,-2.559118e-17,-3.838678e-17,0.000000,-2.559118e-17,3.838678e-17,12041.957635,0.5
std,164149.486121,1.000001e+00,1.000001e+00,1.000001e+00,1.000001,1.000001e+00,1.000001e+00,1.000001,1.000001,1.000001,1.000001e+00,1.000001,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001,1.000001,1.000001,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001,1.000001e+00,1.000001e+00,6919.644449,0.5
min,0.000000,-3.495584e+00,-4.996657e+01,-3.183760e+00,-4.951222,-9.952786e+00,-2.111111e+01,-4.351839,-10.756342,-3.751919,-3.163276e+00,-5.954723,-2.020399e+00,-5.955227e+00,-2.107417e+00,-3.861813e+00,-2.214513e+00,-2.484938,-2.421949,-7.804988,-7.814784e+01,-1.938252e+01,-7.734798e+00,-3.029545e+01,-4.067968e+00,-1.361263e+01,-8.226969,-1.049863e+01,-3.903524e+01,50.010000,0.0
25%,142157.250000,-5.652859e-01,-4.866777e-01,-6.492987e-01,-0.656020,-2.934955e-01,-4.458712e-01,-0.283533,-0.192257,-0.568745,-5.901008e-01,-0.701449,-8.311331e-01,-6.966667e-01,-8.732057e-01,-6.212485e-01,-7.162655e-01,-0.619491,-0.556046,-0.565308,-3.502399e-01,-1.664408e-01,-4.904892e-01,-2.376289e-01,-6.515801e-01,-5.541485e-01,-0.631895,-3.049607e-01,-2.318783e-01,6054.892500,0.0
50%,284314.500000,-9.363846e-02,-1.358939e-01,3.528579e-04,-0.073762,8.108788e-02,7.871758e-02,0.233366,-0.114524,0.092526,2.626145e-01,-0.041050,1.620521e-01,1.760812e-02,2.305011e-01,-3.925566e-02,1.340262e-01,0.271641,0.087294,-0.025979,-1.233776e-01,-3.743065e-02,-2.732881e-02,-5.968903e-02,1.590123e-02,-8.193162e-03,-0.011892,-1.729111e-01,-1.392973e-02,12030.150000,0.5
75%,426471.750000,8.326582e-01,3.435552e-01,6.285380e-01,0.707005,4.397368e-01,4.977881e-01,0.525955,0.047299,0.559262,5.924603e-01,0.747773,7.446723e-01,6.856048e-01,7.518216e-01,6.654065e-01,6.556061e-01,0.518224,0.544389,0.560116,2.482164e-01,1.479787e-01,4.638817e-01,1.557153e-01,7.007374e-01,5.500147e-01,0.672888,3.340230e-01,4.095903e-01,18036.330000,1.0
max,568629.000000,2.229046e+00,4.361865e+00,1.412583e+01,3.201536,4.271689e+01,2.616840e+01,217.873038,5.958040,20.270062,3.172271e+01,2.513573,1.791356e+01,7.187486e+00,1.916954e+01,1.453220e+01,4.665291e+01,6.994124,6.783716,3.831672,2.987281e+01,8.087080e+00,1.263251e+01,3.170763e+01,1.296564e+01,1.462151e+01,5.623285,1.132311e+02,7.725594e+01,24039.930000,1.0


In [7]:
fraud = train_df[train_df['Class'] == 1]
non_fraud = train_df[train_df['Class'] == 0]

In [8]:
print(f"Total records in dataset is : {train_df.shape[0]} and columns are {train_df.shape[1]}")
print(f"Total fraud records are : {len(fraud)}")
print(f"Total non=fraud records are : {len(non_fraud)}")

Total records in dataset is : 568630 and columns are 31
Total fraud records are : 284315
Total non=fraud records are : 284315


# - 2.2 **Observations**

- Dataset does not have null values
- Dataset have all the values in numeric/ decimal format
- Because the records count is bit high, we will use sampling technique in ML, to train the model and this will also tells that how the data behaves with different models.
- For Deep Learning (NN), we will directly use this data because NN works well on large datasets. Will convert data into train/ test with shuffle split so that our target values of 0 and 1 will get equally split in both. With this training and testing datasets will have right data.

**NOTE : Lets split the data and take 1% of data seperate. Will check model accuracy after training with this. This data will not be envolve at any phase and will be used for prediction only.**

In [9]:
# 0.1% of the data
percentage = 0.1 / 100
subset_size = int(len(train_df) * percentage)

data_shuffled = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

data_for_prediction = data_shuffled.iloc[:subset_size]  # 0.1% of the data
remaining_data = data_shuffled.iloc[subset_size:]  # The rest of the data

print(f"Total rows: {len(train_df)}")
print(f"Subset size (0.1%): {len(data_for_prediction)}")
print(f"Remaining data size: {len(remaining_data)}")

Total rows: 568630
Subset size (0.1%): 568
Remaining data size: 568062


# - 2.3 **Observations**:

- Finally we have 568062 records for further use and 568 records are to check the model performce. We will use these records after our model got train and tested on testing data.

# 3. Lets visualize the data and understand all columns.

# - 3.1 This is a dynamic graph. We can zoom in/out for data understanding.

In [10]:
correlation_matrix = train_df.corr()  #'pearson' 'kendall'

# Create dynamic heatmap using Plotly
fig = px.imshow(
    correlation_matrix,
    text_auto=True,  # Show correlation values on the heatmap
    color_continuous_scale='Viridis',  # Choose a color scale
    title='Correlation Heatmap of Credit Card Fraud Dataset',
    labels=dict(x="Features", y="Features", color="Correlation")
)

fig.update_layout(
    width=1250,
    height=900,
    margin=dict(l=5, r=5, t=50, b=30),  # padding around the plot
    xaxis=dict(tickangle=45),  # Rotate x-axis labels
    font=dict(size=14)  # Adjust font size
)

fig.show()

# - 3.2 Observation

- With the above graph we get to know that more the green color (near to 1) having more strong relation with Class column.

# 4. Split data

- Split data based on Class so that values of Class like 0's and 1's will get split equally in train and test datasets.

In [11]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Perform the split
for train_index, test_index in sss.split(train_df, train_df['Class']):
    train_set = train_df.iloc[train_index]
    test_set = train_df.iloc[test_index]

- Data preprations

In [12]:
X = train_set.drop(['id','Class'], axis=1)
y = train_set['Class']

test_X = test_set.drop(['id','Class'], axis=1)
test_y = test_set['Class']

In [13]:
X.shape, y.shape,test_X.shape,test_y.shape

((454904, 29), (454904,), (113726, 29), (113726,))

# - 4.1 Observations:

- Finally we have 454904 records to train our model with 29 columns
- 113726 records for model testing.

In [14]:
scaler = StandardScaler()
train_features = scaler.fit_transform(X)
test_features = scaler.transform(test_X)

# 5. Model training, testing and checking accuracy

- **Neural Networks**

# - 5.1 Hyperparams for NN

- **dropout_layer1** : Dropout randomly disables a fraction of neurons during training, forcing the network to learn more robust features. This reduces overfitting(big advantage). This drops that much amount % of neurons.
- **tf.keras.regularizers.l2** : Add L1 or L2 regularization to the Dense layers to penalize large weights, which can help reduce overfitting.
- **hidden_layer1** : try changing number of neurons. It changes the model performace and also reduce the overfitting.
- **Data Normalization Layer** : *batch_norm_layer = tf.keras.layers.BatchNormalization()*  Batch normalization normalizes the input to each layer, stabilizing training and improving generalization.
- **EarlyStopping** : This stops our training to do not run on extra epochs. So saves our time and overfitting also. Use the below 2 techniques for better results:
  - restore_best_weights=True  #to ensure the model reverts to the best weights observed during training
  - monitor='val_loss'   ## Monitor validation loss


In [15]:
# Input Layer
input_layer = tf.keras.layers.Dense(64, activation='relu', input_shape=(train_features.shape[1],))

# Hidden Layers with Regularization
hidden_layer1 = tf.keras.layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.02))
batch_norm1 = tf.keras.layers.BatchNormalization()
activation1 = tf.keras.layers.LeakyReLU(alpha=0.1)
dropout1 = tf.keras.layers.Dropout(0.4)

hidden_layer2 = tf.keras.layers.Dense(16, kernel_regularizer=tf.keras.regularizers.l2(0.02))
batch_norm2 = tf.keras.layers.BatchNormalization()
activation2 = tf.keras.layers.LeakyReLU(alpha=0.1)
dropout2 = tf.keras.layers.Dropout(0.4)

hidden_layer3 = tf.keras.layers.Dense(8, kernel_regularizer=tf.keras.regularizers.l2(0.02))
batch_norm3 = tf.keras.layers.BatchNormalization()
activation3 = tf.keras.layers.LeakyReLU(alpha=0.1)

# Output Layer
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

# Build the Model
model = tf.keras.Sequential([
    input_layer,

    hidden_layer1, batch_norm1, activation1, dropout1,
    hidden_layer2, batch_norm2, activation2, dropout2,
    hidden_layer3, batch_norm3, activation3,

    output_layer
])

early_stopping = EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss')

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')  # AUC for better fraud detection evaluation
    ]
)
records = model.fit(train_features, y, epochs=10, batch_size=32, verbose=1,
                    validation_data=(test_features, test_y), callbacks=[early_stopping])
print("Done")
# loss, accuracy = model.evaluate(test_features, test_y, verbose=1)
# print(f"Test Loss: {loss:.4f}")
# print(f"Test Accuracy: {accuracy:.4f}")

# plt.plot(records.history['loss'])
# plt.plot(records.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper right')
# plt.show()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning:

Argument `alpha` is deprecated. Use `negative_slope` instead.



Epoch 1/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 43s 3ms/step - accuracy: 0.9632 - auc: 0.9902 - loss: 0.2310 - precision: 0.9748 - recall: 0.9506 - val_accuracy: 0.9925 - val_auc: 0.9992 - val_loss: 0.0507 - val_precision: 0.9943 - val_recall: 0.9907
Epoch 2/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.9899 - auc: 0.9989 - loss: 0.0548 - precision: 0.9904 - recall: 0.9894 - val_accuracy: 0.9965 - val_auc: 0.9996 - val_loss: 0.0355 - val_precision: 0.9956 - val_recall: 0.9975
Epoch 3/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.9925 - auc: 0.9992 - loss: 0.0462 - precision: 0.9928 - recall: 0.9923 - val_accuracy: 0.9974 - val_auc: 0.9995 - val_loss: 0.0350 - val_precision: 0.9964 - val_recall: 0.9984
Epoch 4/10
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.9936 - auc: 0.9992 - loss: 0.0441 - precision: 0.9934 - recall: 0.9937 - val_accuracy: 0.9981 - val_auc: 0.9996 - val_loss: 0.0282 - val_precision: 0.9964 - val_recall: 0.9999
Epoch 5/10
1

In [31]:
results = model.evaluate(test_features, test_y, verbose=1)
loss, accuracy, precision, recall, f1_score = results  # Adjust variable names accordingly

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1_score:.4f}")


3554/3554 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9988 - auc: 0.9998 - loss: 0.0206 - precision: 0.9981 - recall: 0.9995
Test Loss: 0.0211
Test Accuracy: 0.9987
Test Precision: 0.9977
Test Recall: 0.9996
Test F1-Score: 0.9998


In [23]:
print(results)

[0.021060306578874588, 0.9986546635627747, 0.997718095779419, 0.9995955228805542, 0.9997815489768982]


# Output results:

- As we can see that model accuracy is almost 100% and loss is very less. That means our model is perfectly build.
- This graph tells us that how our epochs were keep on running and making our model best till, it did not found that the prediction results are similar as validation dataset.

# 6. Check results on unseen data by applying this model.

**Lets test this model with our pre-declared prediction data.**

In [24]:
data_for_prediction.head(20)

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,437378,0.420468,-0.070194,-0.569266,0.191673,-0.009607,0.426903,-0.356728,0.096143,0.077806,-0.673283,0.586506,-0.750942,-1.033356,-0.886895,0.543423,-1.075065,-0.665189,-1.149084,-0.168782,0.235116,0.134969,0.070433,0.047770,-0.851622,0.102876,-0.375436,0.820807,0.665983,8633.18,1
1,504222,-0.238944,0.250929,-0.374408,0.152938,-0.105008,-0.039028,-0.293004,0.133771,-0.591631,-0.651518,0.354661,-0.744777,0.331696,-0.731682,0.603739,-1.053119,-0.787556,-1.092381,0.895043,0.332143,0.192405,0.289441,-0.255187,-0.817462,0.308284,1.582688,0.574425,0.478489,12299.55,1
2,4794,-0.117796,-0.147961,2.130455,-0.325762,0.325616,0.271351,0.772625,-0.244342,1.240012,0.414358,-0.310268,-0.126386,2.950418,1.211172,-0.858423,0.055065,0.779386,0.093685,0.513854,0.110696,-0.271739,-0.404654,-0.121235,0.857659,0.541920,0.756534,-0.238177,-0.403038,5215.87,0
3,388411,-0.855315,0.137014,-0.628116,0.613733,-0.643573,-0.664283,-0.880040,0.466586,-1.045508,-1.234377,1.133538,-1.177124,0.349663,-1.254409,-1.237353,-1.409580,-1.626020,-1.458452,1.084589,0.552311,0.405505,0.167560,0.446262,-0.205976,0.492582,0.658619,1.609128,-0.025592,19282.98,1
4,424512,0.257686,0.035247,-0.203112,0.506745,-0.242235,-0.192608,-0.289297,0.044488,-0.396122,-0.370119,0.874225,-0.730394,-2.110598,-0.634357,0.440263,-0.595804,-0.735223,-0.463177,-0.325932,0.124330,0.162191,0.165912,-0.181999,0.331451,1.043095,0.029799,0.643273,0.736723,19114.27,1
5,123536,-0.413357,-0.450377,1.184996,-0.378135,1.009580,1.385973,0.525802,-0.026273,0.428828,0.614420,0.073380,1.288058,-0.602212,0.744269,-0.010914,-0.188847,0.734558,-0.497864,-0.947582,-0.954756,-0.139289,0.633786,0.356072,-2.138775,-0.093777,-0.721594,-0.664833,-1.516420,12622.46,0
6,333319,-0.905109,1.090565,-1.187502,1.927398,-1.209692,-1.471852,-1.480131,0.820961,-1.747814,-1.673500,1.844200,-1.919698,0.157106,-1.716313,0.394367,-2.020690,-2.172109,-2.032614,1.907355,0.859102,0.757018,0.582996,0.269935,0.962761,-0.860951,0.885863,1.626802,1.396159,21616.06,1
7,369666,0.517429,-0.016506,-0.225822,0.437507,0.045921,0.111977,-0.120557,-0.011100,-0.381664,-0.149901,0.482652,-0.580711,-1.199153,-0.604071,0.081929,0.011340,-0.446074,0.139542,-0.260829,0.131684,0.071712,-0.060887,-0.281729,-1.003867,1.059399,0.416818,0.365278,0.537678,2518.11,1
8,62882,-0.330014,-0.161835,1.288293,-0.591877,0.289209,-0.283643,0.623377,-0.084072,0.080601,0.396721,-0.932837,0.612180,-0.103629,1.097272,0.943783,0.604135,0.405033,-0.035865,0.048024,-0.160212,-0.286617,-1.276665,0.294000,0.687970,0.151482,-0.168056,-0.218770,-0.032723,21222.90,0
9,414847,-1.998296,2.563713,-1.922748,2.220722,-2.315012,0.163451,-2.788664,-1.133662,-2.497692,-2.322209,1.274140,-1.696605,-0.190928,-0.993515,-1.318173,-1.629526,-1.788834,-1.798008,0.873544,2.070130,-2.119779,2.568182,1.578465,0.802100,-1.068481,0.383964,-2.780941,-1.372040,11009.50,1


In [25]:
for_prediction = data_for_prediction.drop(['id','Class'], axis=1)
for_prediction = scaler.transform(for_prediction)

# 7. Save model

In [27]:
 model.save('credit_card_fraud_detect_NN_model.keras')

In [29]:
for i, (record, label) in enumerate(zip(for_prediction, data_for_prediction['Class'])):
    # Reshape the record to match the model input shape (1, num_features)
    record_reshaped = record.reshape(1, -1)
    prediction = model.predict(record_reshaped)
    threshold = 0.5

    predicted_class = (prediction > threshold).astype(int)
    print(f"predicted_class raw data : {predicted_class}")
    print(f"Record {i + 1}:")
    print(f"  Actual Label: {label}")
    print(f"  Scaled Input: {record}")
    print(f"  Raw Prediction (Probability): {prediction[0][0]}")
    print(f"  Predicted Class: {predicted_class[0][0]}")
    print("-" * 50)
    break  #remove this break, if you want to see that how prediction gets match on all records of unseen data.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
predicted_class raw data : [[1]]
Record 1:
  Actual Label: 1
  Scaled Input: [ 0.4201251  -0.07010059 -0.56946044  0.19167551 -0.00950122  0.42721494
 -0.35446989  0.0962281   0.07782132 -0.67408304  0.58670638 -0.75131322
 -1.03386924 -0.88721337  0.54294162 -1.0746347  -0.66493438 -1.14827143
 -0.1682811   0.2357356   0.13526966  0.07009707  0.0474236  -0.85254013
  0.1031686  -0.37536891  0.81896055  0.66655887 -0.49244815]
  Raw Prediction (Probability): 0.9967625141143799
  Predicted Class: 1
--------------------------------------------------
